In [1]:
import subprocess
import sys
import time
import nbimporter

In [2]:
import utils as utils
import blockchain as blockchain_utils
import device as device_utils
import oracle as oracle_software

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb
Importing Jupyter notebook from device.ipynb
Importing Jupyter notebook from oracle.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
latest = utils.load_json('resources/latest.json')

In [5]:
device_info = utils.load_yaml('resources/identifier.yaml')

### CONNECT TO ETHEREUM GATEWAY

In [6]:
web3 = blockchain_utils.connect(settings)

### EXTRACT THE WHISPER API

In [7]:
shh = web3.geth.shh

### GENERATE A FRESH WHISPER ID

In [8]:
whisper_id = shh.newKeyPair()

### CREATE DEVICE OUTLINE

In [9]:
oracle = device_utils.create(device_info)

### SERIALIZE NECESSARY MANAGER CONTRACTS

In [10]:
oracle_manager = blockchain_utils.contract(latest['oraclemanager'], web3, settings)

In [11]:
task_manager = blockchain_utils.contract(latest['taskmanager'], web3, settings)

### FETCH & SERIALIZE ORACLE CONTRACT

In [12]:
temp_contract = blockchain_utils.contract({
    'address': oracle_manager.read({
        'func': 'fetch_oracle',
        'params': oracle.hash
    }),
    'abi': latest['oracle']['abi']
}, web3, settings)

### VERIFY ORACLE CONTRACT EXISTENCE

In [13]:
if temp_contract.address != '0x0000000000000000000000000000000000000000':
    oracle.set_contract(temp_contract)
    
else:
    utils.log('THE ORACLE IS NOT REGISTERED, ABORTING...')
    sys.exit(0)

### GLOBAL TASK BACKLOG

In [14]:
raw = oracle.read('fetch_backlog')

In [15]:
backlog = utils.filter_backlog(raw)

### GLOBAL ACTIVE STATUS 

In [16]:
active = oracle.read('active')

### GLOBAL DISCOVERY STATUS

In [17]:
discoverable = oracle.read('discoverable')

### GLOBAL DISCOVERY CONFIG

In [18]:
encoded = oracle.read('config')

In [19]:
discovery_config = utils.decode(encoded)

### UDATE DEVICE STATUS & DETAILS

In [20]:
def update_details():
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global discovery_config
    global backlog
    
    # EXTRACT RELEVANT VALUES
    latest_active = oracle.read('active')
    latest_discoverable = oracle.read('discoverable')
    latest_config = utils.decode(oracle.read('config'))

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        utils.log('ACTIVE STATUS CHANGED TO: ' + str(latest_active) + '\n')
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        utils.log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable) + '\n')
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        utils.log('DISCOVERY CONFIG CHANGED\n')
        
    # UPDATE BACKLOG
    raw_backlog = oracle.read('fetch_backlog')
    backlog = utils.filter_backlog(raw_backlog)

### UPDATE MIDDLEWARE

In [21]:
def update_middleware():
    
    # PRINT REACTION
    utils.log('MIDDLEWARE UPDATE TRIGGERED\n')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

### PERFORM TASK

In [22]:
def perform_task(task):
    
    # SHOW MSG
    utils.log('STARTING TASK: ' + task)
    
    # SERIALIZE THE TASK CONTRACT
    task_contract = blockchain_utils.contract({
        'address': task,
        'abi': latest['task']['abi']
    }, web3, settings)
    
    # FETCH & DECODE TASK PARAMS
    params = task_contract.read('params')
    decoded = utils.decode(params)

    # PERFORM ORACLE TASK
    result = oracle_software.perform_task(task, decoded)

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, result]
    })

    # SHOW MSG
    utils.log('TASK COMPLETED\n')

### DISCOVERY RESPONSES

In [23]:
def process_message(event):
    
    # SERIALIZE EVENT PARAMS
    author = web3.toHex(event['sig'])
    payload = web3.toText(event['payload'])

    # DECODE THE PAYLOAD
    data = utils.decode(payload)
    
    # REQUIRED KEYS FOR VALID MESSAGE
    required = ['type', 'discovery']

    # DECODED KEYS
    keys = list(data.keys())
    
    # THE REQUEST KEYWORD FOR THE PAYLOAD TYPE
    keyword = 'request'
    
    # IF THE KEYSETS MATCH & THE TYPE IS A REQUEST
    if (required == keys and data['type'] == keyword):
        
        # CHECK MATCHES IN DISCOVERY PARAMS
        discovery_result = utils.compare_discovery(data['discovery'], discovery_config)
        
        # IF EVERYTHING MATCHED
        if (discovery_result.count(False) == 0):
            
            # SHOW MSG
            utils.log('DISCOVERY REQUEST DETECTED\n')
            
            # ENCODE A JSON RESPONSE
            response = utils.encode({
                'type': 'response',
                'source': payload,
                'oracle': oracle.hash
            })
            
            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['symkey'],
                'payload': web3.toHex(text=response),
                'topic': web3.toHex(text=settings['whisper']['topic']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### CONTRACT EVENTS

In [24]:
update_event = oracle.event('middleware')

In [25]:
modification_event = oracle.event('modification')

In [26]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']),
    'symKeyID': settings['whisper']['symkey']
})

### EVENT LOOP

In [27]:
try:
    utils.log('AWAITING EVENTS...\n')
    
    while(True):

        # VARIABLE MODIFICATION EVENT
        for event in modification_event.get_new_entries():
            update_details()

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)
        
# KILL PROCESS
except:
    utils.log('THE PROCESS WAS MANUALLY KILLED!')

[LAUNCHER] AWAITING EVENTS...

[LAUNCHER] MIDDLEWARE UPDATE TRIGGERED

[LAUNCHER] THE PROCESS WAS MANUALLY KILLED!
